# Steganografia - testy narzędzi
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb](https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł
- Anton Maisiuk

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - Ocena narzędzi steganograficznych 

Na podstawie listy ze strony [Wikipedii](https://en.wikipedia.org/wiki/Steganography_tools) wybierz 5 narzędzi steganograficznych i dokonaj ich oceny (badań):
- porównaj szybkość działania dla tego samego pliku,
- jeżeli to możliwe, to oceń ilość/jakość zmian pliku źródłowego.


## Zadanie 2 - Własny system steganograficzny

Wykonaj prosty system steganograficzny plików graficznych BMP polegający na osadzaniu w najmniej znaczących bitach danego stegotekstu:
- ilość bitów ma być zmienna, od 1 do 4,
- dokonaj oceny wizualnej głębokości zmian pliku źródłowego w zależności od ilości bitów modyfikowanych.

Najpierw została zadeklarowana funkcja pomocnicza `getBitsIterator` zwracająca iterator, który ułatwia przeglądanie tablicy bajtów po wybranej ilości bitów (1-4). Funkcjonalność ta ułatwi podizelenie danych na małe fragmenty, które zostaną osadzone w najmniej znaczących częściach bajtów.

In [54]:
function* getBitsIterator(bytes: Uint8Array, bits: 1 | 2 | 3 | 4) {
    switch (bits) {
        case 1:
            for (const byte of bytes) {
                yield (byte & 0b10000000) >> 7;
                yield (byte & 0b01000000) >> 6;
                yield (byte & 0b00100000) >> 5;
                yield (byte & 0b00010000) >> 4;
                yield (byte & 0b00001000) >> 3;
                yield (byte & 0b00000100) >> 2;
                yield (byte & 0b00000010) >> 1;
                yield (byte & 0b00000001);
            }
            break;
        case 2:
            for (const byte of bytes) {
                yield (byte & 0b11000000) >> 6;
                yield (byte & 0b00110000) >> 4;
                yield (byte & 0b00001100) >> 2;
                yield (byte & 0b00000011);
            }
            break;
        case 3:
            {
                // Holds the current bits buffer
                let buffer = 0; 
                 // Number of bits available in the buffer
                let bufferLength = 0;

                for (const byte of bytes) {
                    // Add new byte to the buffer
                    buffer = (buffer << 8) | byte;
                    bufferLength += 8;

                    // Emit chunks as long as we can get 3 bits
                    while (bufferLength >= 3) {
                        yield (buffer >> (bufferLength - 3)) & 0b111; 
                        bufferLength -= 3;
                    }
                }

                // Handle leftover bits (if any) by padding with zeros
                if (bufferLength > 0) {
                    yield (buffer << (3 - bufferLength)) & 0b111; // Pad with zeros
                }
            }
            break;
        case 4:
            for (const byte of bytes) {
                yield (byte & 0b11110000) >> 4;
                yield (byte & 0b00001111);
            }
            break;
    }
}

Następnie została zadeklarowana funkcja `embedText`. Przyjmuje ona bufor zawierający zawartość pliku wejściowego, tekst do osadzenia oraz liczbę bitów na bajt, w których zostanie osadzony tekst. Program wykorzystuje bibliotekę `bmp-js`. Do dekodowania i enkodowania obrazów w formacie `bmp`. Koniec tekstu jest oznaczany symbolem specjalnym `\0`.

In [55]:
//@deno-types="npm:@types/bmp-js"
import { decode, encode } from "npm:bmp-js";
import type { Buffer } from "node:buffer";

function embedText(inputFile: Buffer, text: string, bits: 1 | 2 | 3 | 4) {
    // decode bmp data
    const bmpData = decode(inputFile);
    // encode text to binary using 'utf-8'
    const textEncoder = new TextEncoder();
    const textBytes = textEncoder.encode(text + "\0");
    // create bits iterator for easier splitting into chunks
    const bitsIterator = getBitsIterator(
        textBytes,
        bits,
    );
    let i = 0;
    const mask = 255 << bits;
    for (const chunk of bitsIterator) {
        // don't write to alpha channel
        if (!(i % 4)) i++;
        // save chunk into least significant bits
        bmpData.data[i] = (bmpData.data[i] & mask) | chunk;
        i++;
    }
    // endode bmp data back 
    return encode(bmpData);
}

Funkcja `readEmbeddedText` działa analogicznie. Odczytuje tekst osadzony w pliku. Funkcja odczytuje tekst aż do natrafienia na specjalny symbol kończący `\0`. Jest on usuwany z końcowego wyniku.

In [ ]:
function readEmbeddedText(inputFile: Buffer, bits: 1 | 2 | 3 | 4) {
    // decode bmp data
    const bmpData = decode(inputFile);
    // declare mask for reading only bits with embedded data
    const mask = 255 >> (8 - bits);

    const readBytes: number[] = [];
    let chunkBuffer = 0;
    let chunkBufferLength = 0;
    let i = -1;
    for (const byte of bmpData.data) {
        // don't read from alpha channel
        if (!(++i % 4)) continue;
        // read embedded data from byte
        const chunk = byte & mask;
        // save read data to buffer
        chunkBuffer = chunkBuffer << bits;
        chunkBuffer = chunkBuffer | chunk;
        chunkBufferLength += bits;

        // if there is already full byte in buffer
        if (chunkBufferLength >= 8) {
            //get full byte
            const overflowBits = chunkBufferLength - 8;
            const fullByte = chunkBuffer >> overflowBits;

            //left in buffer only overflowed bits
            chunkBuffer &= 255 >> (8 - overflowBits);
            chunkBufferLength = overflowBits;

            //write fullbyte to result array
            readBytes.push(fullByte);

            // end if read character is '/0'
            if (fullByte === 0) {
                break;
            }
        }
    }

    // decode text from binary utf-8 and remove end character
    const textDecoder = new TextDecoder();
    return textDecoder.decode(new Uint8Array(readBytes)).slice(0, -1);
}

Do przetestowania programu posłuży zdjęcie kota, w którym zostanie osadzony krótki tekst. Wyniki prezentuje poniższa tabela.

In [ ]:
import fs from "node:fs";

const bmpBuffer = fs.readFileSync("cat.bmp");
const inputText = "To jest pewien tekst zakodowany w obrazie. Czy widać jakąś różnicę?"
const bmpModified = embedText(
    bmpBuffer,
    inputText,
    3,
);

fs.writeFileSync("catModified.bmp", bmpModified.data);
const readFile = fs.readFileSync("catModified.bmp");
const readText = readEmbeddedText(readFile, 3);

Deno.jupyter.md`
|Tekst wejściowy| ${inputText} |
|-|-|
|Obraz wejściowy| ![](cat.bmp) |
|Obraz z osadzonym tekstem|  ![](catModified.bmp) |
|Odczytany tekst| ${readText} |
`



|Tekst wejściowy| To jest pewien tekst zakodowany w obrazie. Czy widać jakąś różnicę? |
|-|-|
|Obraz wejściowy| ![](cat.bmp) |
|Obraz z osadzonym tekstem|  ![](catModified.bmp) |
|Odczytany tekst| To jest pewien tekst zakodowany w obrazie. Czy widać jakąś różnicę? |


Do wizualnego porównania zdjęcia dla różnych długości bitów będzie potrzebny dłuższy tekst. W tym celu pobrany został skrypt filmu pod tytułem "Film o pszczołach".

In [ ]:
const beeMovieScript = await fetch("https://gist.githubusercontent.com/MattIPv4/045239bc27b16b2bcf7a3a9a4648c08a/raw/2411e31293a35f3e565f61e7490a806d4720ea7e/bee%2520movie%2520script").then(r => r.text())
beeMovieScript.substring(0, 100) + "(...)";

"According to all known laws of aviation, there is no way a bee should be able to fly.\n" +
  "Its wings are (...)"

Tekst zostanie zapisany w obrazie dla długości 1, 2, 3 i 4 bitów. Wyniki zostaną wyświetlione w tabeli.

In [ ]:
const bmpOriginalBuffer = fs.readFileSync("cat.bmp");
for (let i = 1; i <=4; i++) {
    const bmpModified = embedText( bmpOriginalBuffer, beeMovieScript, i);
    fs.writeFileSync(`cat${i}.bmp`, bmpModified.data);
}

Deno.jupyter.md`
|Obraz wejściowy| ![](cat.bmp) |
|-|-|
|1 bit| ![](cat1.bmp) |
|2 bity| ![](cat2.bmp) |
|3 bity| ![](cat3.bmp) |
|4 bity| ![](cat4.bmp) |
`


|Obraz wejściowy| ![](cat.bmp) |
|-|-|
|1 bit| ![](cat1.bmp) |
|2 bity| ![](cat2.bmp) |
|3 bity| ![](cat3.bmp) |
|4 bity| ![](cat4.bmp) |


Ocena wizualna:
- Dla 1 bitu różnica jest niedostrzealna gołym okiem. Różnica między kolorem oryginalnym to zaledwie 0.4% (dla każdego kanału).
- Dla 2 bitów wprawne oko po przybliżeniu może dostrzec, że pojawił się charakterystyczny szum.
- Dla 3 bitów szum ten jest już dobrze widoczny, zwłaszcza na fragmentach obrazu o jednolitym kolorze. W tym przypadku na niebie i powierzchni po prawej stronie obrazu.
- Dla 4 bitów szum ten jest jescze bardziej widoczny. Widać teraz także wyraźnie granicę, gdzie zakodowane dane się skończyły. W celu zmniejszenia podejrzeń o istnieniu osadzonych danych, warto jest zakodować tekst na całej powierzchni obrazu.